# Trial with Graph Convolutional Network

In [1]:
import os
os.environ["OMP_NUM_THREADS"] = "1"

## 方針

### 教師あり学習

まずは教師あり学習により強いポリシーを獲得。

### 自己対戦による訓練

* Q関数が相手によって変わってしまう。
    * On-policyで自己対戦にすればOK。Off-policyにしたい場合は、Q関数が大きく違わない過去エピソードとすべき。
        * Rainbow
            * PFRLに実装あり。まずはこれ？
        * MuZero
            * 実装少し大変かも。だが自己対戦による実績あるため性能は出るかも？
    * Policyを持つアルゴリズムならOK。相手ごとにQ関数を学習すれば良い。この場合も自己対戦かつOn-policy。
        * AC3、PPOなど
            * PFRLに実装あり。これもトライ？
            
### 方策／Q関数モデル

* Graph Neural Network
    * あたかもそれぞれの選手が行動判断／価値判断しているようなモデルにする。選択されたアクションは、その時Activeな選手の最善手とする。
    * 初めは、特徴量は、絶対位置座標で、完全グラフを用いる。

## Actions

### Default action set

The default action set consists of 19 actions:

*   Idle actions

    *   `action_idle` = 0, a no-op action, stickly actions are not affected (player maintains his directional movement etc.).

*   Movement actions

    *   `action_left` = 1, run to the left, sticky action.
    *   `action_top_left` = 2, run to the top-left, sticky action.
    *   `action_top` = 3, run to the top, sticky action.
    *   `action_top_right` = 4, run to the top-right, sticky action.
    *   `action_right` = 5, run to the right, sticky action.
    *   `action_bottom_right` = 6, run to the bottom-right, sticky action.
    *   `action_bottom` = 7, run to the bottom, sticky action.
    *   `action_bottom_left` = 8, run to the bottom-left, sticky action.

*   Passing / Shooting

    *   `action_long_pass` = 9, perform a long pass to the player on your team. Player to pass the ball to is auto-determined based on the movement direction.
    *   `action_high_pass` = 10, perform a high pass, similar to `action_long_pass`.
    *   `action_short_pass` = 11, perform a short pass, similar to `action_long_pass`.
    *   `action_shot` = 12, perform a shot, always in the direction of the opponent's goal.

*   Other actions

    *   `action_sprint` = 13, start sprinting, sticky action. Player moves faster, but has worse ball handling.
    *   `action_release_direction` = 14, reset current movement direction.
    *   `action_release_sprint` = 15, stop sprinting.
    *   `action_sliding` = 16, perform a slide (effective when not having a ball).
    *   `action_dribble` = 17, start dribbling (effective when having a ball), sticky action. Player moves slower, but it is harder to take over the ball from him.
    *   `action_release_dribble` = 18, stop dribbling.

### V2 action set

It is an extension of the default action set:

*   `action_builtin_ai` = 19, let game's built-in AI generate an action

In [2]:
# Install:
# Kaggle environments.
!git clone https://github.com/Kaggle/kaggle-environments.git
!cd kaggle-environments && pip install .

# GFootball environment.
!apt-get update -y
!apt-get install -y libsdl2-gfx-dev libsdl2-ttf-dev

# Make sure that the Branch in git clone and in wget call matches !!
!git clone -b v2.3 https://github.com/google-research/football.git
!mkdir -p football/third_party/gfootball_engine/lib

!wget https://storage.googleapis.com/gfootball/prebuilt_gameplayfootball_v2.3.so -O football/third_party/gfootball_engine/lib/prebuilt_gameplayfootball.so
!cd football && GFOOTBALL_USE_PREBUILT_SO=1 pip3 install .

fatal: destination path 'kaggle-environments' already exists and is not an empty directory.
Processing /notebooks/kaggle/gfootball/kaggle-environments
  Created wheel for kaggle-environments: filename=kaggle_environments-1.3.14-py3-none-any.whl size=302295 sha256=4ff1eab431d2a148af1b404dce2ce65c241d7b6dbb393a1d65fb8d067435f101
  Stored in directory: /root/.cache/pip/wheels/32/ff/b6/a9ab62cd5f60b2492aa5d5bc96a6d12bb1158496e87a4576ec
Successfully built kaggle-environments
  Attempting uninstall: kaggle-environments
    Found existing installation: kaggle-environments 1.3.14
    Uninstalling kaggle-environments-1.3.14:
      Successfully uninstalled kaggle-environments-1.3.14
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease                       
Ign:4 http://developer.download.nvidia.com/compute/machine-learning/repo

## Install

In [3]:
!pip install pfrl==0.1.0

In [4]:
# ------------------ install torch_geometric begin -----------------
try:
    import torch_geometric
except:
    import subprocess
    import torch

    nvcc_stdout = str(subprocess.check_output(['nvcc', '-V']))
    tmp = nvcc_stdout[nvcc_stdout.rfind('release') + len('release') + 1:]
    cuda_version = tmp[:tmp.find(',')]
    cuda = {
            '9.2': 'cu92',
            '10.1': 'cu101',
            '10.2': 'cu102',
            }

    CUDA = cuda[cuda_version]
    TORCH = torch.__version__.split('.')
    TORCH[-1] = '0'
    TORCH = '.'.join(TORCH)

    install1 = 'pip install torch-scatter==latest+' + CUDA + ' -f https://pytorch-geometric.com/whl/torch-' + TORCH + '.html'
    install2 = 'pip install torch-sparse==latest+' + CUDA + ' -f https://pytorch-geometric.com/whl/torch-' + TORCH + '.html'
    install3 = 'pip install torch-cluster==latest+' + CUDA + ' -f https://pytorch-geometric.com/whl/torch-' + TORCH + '.html'
    install4 = 'pip install torch-spline-conv==latest+' + CUDA + ' -f https://pytorch-geometric.com/whl/torch-' + TORCH + '.html'
    install5 = 'pip install torch-geometric'

    subprocess.run(install1.split())
    subprocess.run(install2.split())
    subprocess.run(install3.split())
    subprocess.run(install4.split())
    subprocess.run(install5.split())
# ------------------ install torch_geometric end -----------------

In [73]:
import os
import cv2
import sys
import glob 
import random
import json
import pickle
import copy
import imageio
import pathlib
import collections
from collections import deque, namedtuple
import numpy as np
import pandas as pd
import argparse
from IPython.display import clear_output
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import Parallel, delayed
sns.set()
%matplotlib inline

from gym import spaces
from tqdm import tqdm
from logging import getLogger, StreamHandler, FileHandler, DEBUG, INFO
from typing import Union, Callable, List, Tuple, Iterable, Any, Dict
from dataclasses import dataclass
from IPython.display import Image, display
sns.set()


# PFRL
import pfrl
from pfrl.agents import CategoricalDoubleDQN
from pfrl import experiments
from pfrl import explorers
from pfrl import nn as pnn
from pfrl import utils
from pfrl import replay_buffers
from pfrl.wrappers import atari_wrappers
from pfrl.q_functions import DistributionalDuelingDQN

# PyTorch
import torch
from torch import nn
import torch.multiprocessing as mp
import torch.nn.functional as F
import torch.distributions as D

# PyTorch geometric
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import RGCNConv
from torch_geometric.data import InMemoryDataset

from torch_scatter import scatter_max, scatter_sum, scatter_mean

# Env
import gym
import gfootball
import gfootball.env as football_env
from gfootball.env import observation_preprocessing
from gfootball.env.wrappers import Simple115StateWrapper

## Config

In [6]:
# Check we can use GPU
print(torch.cuda.is_available())

# set gpu id
if torch.cuda.is_available(): 
    # NOTE: it is not number of gpu but id which start from 0
    gpu = 0
else:
    # cpu=>-1
    gpu = -1

True


In [7]:
# set logger
def logger_config():
    logger = getLogger(__name__)
    handler = StreamHandler()
    handler.setLevel("DEBUG")
    logger.setLevel("DEBUG")
    logger.addHandler(handler)
    logger.propagate = False

    filepath = './result.log'
    file_handler = FileHandler(filepath)
    logger.addHandler(file_handler)
    return logger

logger = logger_config()

In [8]:
# fixed random seed
# but this is NOT enough to fix the result of rewards.Please tell me the reason.
def seed_everything(seed=1234):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    utils.set_random_seed(seed)  # for PFRL
    
# Set a random seed used in PFRL.
seed = 5046
seed_everything(seed)

# Set different random seeds for train and test envs.
train_seed = seed
test_seed = 2 ** 31 - 1 - seed

## Environment

In [9]:
env = football_env.create_environment(
    env_name='11_vs_11_kaggle',  # easy mode
    stacked=False,
    representation='simple115v2',           # SMM
    rewards='scoring, checkpoints',
    write_goal_dumps=False,
    write_full_episode_dumps=False,
    render=False,
    write_video=False,
    dump_frequency=1,
    logdir='./',
    extra_players=None,
    number_of_left_players_agent_controls=1,
    number_of_right_players_agent_controls=1,
)

In [10]:
array = env.reset()

In [11]:
obs, _, _, info = env.step([1,1])

## GCN-Policy

The flag to distinguish left players, right players, and the ball.

* left players: 0
* right players: 1
* ball: 2

In [12]:
relations_dict = {
    (0., 0.): 0, # left player -> left player
    (0., 1.): 1, # left player -> right player
    (0., 2.): 2, # left player -> ball
    (1., 0.): 3, # right player -> left player
    (1., 1.): 4, # right player -> right player
    (1., 2.): 5, # right player -> ball
    (2., 0.): 6, # ball -> left player
    (2., 1.): 7, # ball -> right player
}

In [13]:
def create_graph_from_observation(state, action=None, reverse=False, reverse_y=False):
    array = state
    
    left_coordinations = np.concatenate([array[:22].reshape(11, 2), np.zeros((11, 1))], axis=-1)
    left_directions = np.concatenate([array[22:44].reshape(11, 2), np.zeros((11, 1))], axis=-1)
    right_coordinations = np.concatenate([array[44:66].reshape(11, 2), np.zeros((11, 1))], axis=-1)
    right_directions = np.concatenate([array[66:88].reshape(11, 2), np.zeros((11, 1))], axis=-1)
    ball_coordination = array[88:91].reshape([1, 3])
    ball_direction = array[91:94].reshape([1, 3])
    ball_ownership = array[94:97] # none, left, right
    active_player = array[97:108].reshape([11, 1])
    game_mode = array[108:]
    
    if reverse:
        left_coordinations[:, 0] *= -1.0
        left_directions[:, 0] *= -1.0
        right_coordinations[:, 0] *= -1.0
        right_directions[:, 0] *= -1.0
        ball_coordination[:, 0] *= -1.0
        ball_direction[:, 0] *= -1.0
        ball_ownership = ball_ownership[[0, 2, 1]]
        active_player = active_player
        game_mode = game_mode

    if reverse_y:
        left_coordinations[:, 1] *= -1.0
        left_directions[:, 1] *= -1.0
        right_coordinations[:, 1] *= -1.0
        right_directions[:, 1] *= -1.0
        ball_coordination[:, 1] *= -1.0
        ball_direction[:, 1] *= -1.0
        ball_ownership = ball_ownership
        active_player = active_player
        game_mode = game_mode

    # Node features
    left_features = np.concatenate([
        0*np.ones((11, 1)),
        left_coordinations,
        left_directions,
        ball_ownership[1]*np.ones((11, 1)),
        active_player,
    ], axis=-1)
    right_features = np.concatenate([
        1*np.ones((11, 1)),
        right_coordinations,
        right_directions,
        ball_ownership[2]*np.ones((11, 1)),
        np.zeros((11, 1)),
    ], axis=-1)
    ball_features = np.concatenate([
        2*np.ones((1, 1)),
        ball_coordination,
        ball_direction,
        np.zeros((1, 1)),
        np.zeros((1, 1)),
    ], axis=-1)

    features = np.concatenate([left_features, right_features, ball_features], axis=0)

    # Edges and relations
    X, Y = np.meshgrid(np.arange(len(features)), np.arange(len(features)))
    all_combinations = np.vstack([X.flatten(), Y.flatten()]).T
    edge_index = np.array(
        [combination for combination in all_combinations if not combination[0] == combination[1]]
    ).T
    types_for_edge_index = features[edge_index][:,:,0]
    relations = [relations_dict[tuple(types)] for types in types_for_edge_index.T]

    # numpy array to torch tensor
    features = torch.tensor(features, dtype=torch.float).contiguous()
    edge_index = torch.tensor(edge_index, dtype=torch.long).contiguous()
    relations = torch.tensor(relations, dtype=torch.long).contiguous()

    if action is None:
        graph = Data(x=features, edge_index=edge_index, edge_type=relations)

    else:
        graph = Data(x=features, edge_index=edge_index, edge_type=relations, y=torch.tensor(action, dtype=torch.long))

    return graph

In [14]:
class Policy(torch.nn.Module):
    def __init__(self, num_features, num_relations):
        super().__init__()
        self.conv1 = RGCNConv(num_features - 2, 256, num_relations=num_relations)
        #self.conv2 = RGCNConv(128, 256, num_relations=num_relations)
        self.fc1 = nn.Linear(256, 256)
        self.fc2 = nn.Linear(256, 19)

    def forward(self, data):
        x, edge_index, edge_type = data.x, data.edge_index, data.edge_type
        flag = x[:,0]
        is_active = x[:,-1]
        x = x[:,1:-1]

        x = self.conv1(x, edge_index, edge_type)
        x = F.relu(x)

        #x = self.conv2(x, edge_index, edge_type)
        #x = F.relu(x)
        
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        
        return D.Categorical(logits=x[is_active.bool()])

In [98]:
class Value(torch.nn.Module):
    def __init__(self, num_features, num_relations):
        super().__init__()
        self.conv1 = RGCNConv(num_features - 2, 256, num_relations=num_relations)
        #self.conv2 = RGCNConv(128, 256, num_relations=num_relations)
        self.fc1 = nn.Linear(256, 256)
        self.fc2 = nn.Linear(256, 1)

    def forward(self, data):
        x, edge_index, edge_type = data.x, data.edge_index, data.edge_type
        flag = x[:,0]
        is_active = x[:,-1]
        x = x[:,1:-1]

        x = self.conv1(x, edge_index, edge_type)
        x = F.relu(x)

        #x = self.conv2(x, edge_index, edge_type)
        #x = F.relu(x)
        
        x = self.fc1(x)
        x = F.relu(x)
        
        if hasattr(data, 'batch'):
            x, argmax = scatter_max(x, data.batch, dim=0)
        else:
            x = torch.max(x, dim=0).values

        x = self.fc2(x)

        return x

## Load initial policy (Supervised)

In [16]:
policy = Policy(num_features=9, num_relations=len(relations_dict))

In [17]:
policy_path = 'policy.pt'
policy.load_state_dict(torch.load(policy_path))

<All keys matched successfully>

## Reinforcement Learning classes and functions

In [18]:
class Episodes:
    def __init__(
        self,
        **kwargs,
    ):
        self.episodes = {
            **kwargs,
        }
        for key in ('state', 'action', 'next_state', 'reward', 'done'):
            if key not in kwargs.keys():
                self.episodes[key] = []
        for key, value in self.episodes.items():
            if len(value) != len(self.episodes['state']):
                raise Exception('The length of {} is invalid.'.format(key))
        
    def append(self, **kwargs):
        for key, value in kwargs.items():
            self.episodes[key].append(value)
        
    def add_new_key(self, key, value):
        if len(value) != self._length:
            raise Exception('The length of {} is invalid.'.format(key))
        elif key in self.episodes.keys():
            raise Exception('The key {} is already defined.'.format(key))
        self.episodes[key] = value
        
    def __len__(self):
        return len(self.episodes['state'])
    
    def __repr__(self):
        return 'Episodes\n* number of episodes: {}\n* number of steps: {}\n* total rewards: {}\n* keys: {}'.format(
            np.sum(self.episodes['done']),
            len(self),
            np.sum(self.episodes['reward']),
            ', '.join(self.episodes.keys()),
        )
    
    def __getitem__(self, key):
        return self.episodes[key]

    def sample(self, size, sequential=True):
        if sequential:
            id_from = random.randint(0, len(self) - size)
            id_to = id_from + size
            sampled_dict = {
                key: value[id_from:id_to]
                for key, value
                in self.episodes.items()
            }
            sampled_episode = Episodes(
                **sampled_dict
            )
            return sampled_episode
        else:
            raise NotImplementedError

#### Episodic functions

In [19]:
def calculate_returns(episodes, gamma, value_net, infinite_horizon=True):
    returns = copy.deepcopy(episodes.episodes['reward'])
    states = episodes.episodes['state']
    dones = episodes.episodes['done']
    for t in reversed(range(len(rews))):
        returns[t] = last_reward_to_go = (
            returns[t] 
            + (1 - dones[t]) * gamma * last_reward_to_go
        )
        if inifinite_horizon and dones[t]:
            returns[t] += value_net(states[t])
    episodes.add_new_key('return', returns)

In [20]:
def calculate_values(episodes, value_net):
    values = [value_net(state) for state in episodes.episodes['state']]
    episodes.add_new_key('value', values)

In [21]:
def concat_episodes(episodes_list):
    keys = episodes_list[0].episodes.keys()
    episodes_dict = {key: [] for key in keys}
    for episodes in episodes_list:
        for key in keys:
            episodes_dict[key] += episodes.episodes[key]
    return Episodes(**episodes_dict)

In [22]:
REVERSE_ACTION = {i: i for i in range(20)}
REVERSE_ACTION[1] = 5
REVERSE_ACTION[5] = 1
REVERSE_ACTION[2] = 4
REVERSE_ACTION[4] = 2
REVERSE_ACTION[6] = 8
REVERSE_ACTION[8] = 6
def reverse_action(action):
    return REVERSE_ACTION[action]

In [23]:
env = football_env.create_environment(
    env_name='11_vs_11_kaggle',  # easy mode
    stacked=False,
    representation='simple115v2',
    rewards='scoring, checkpoints',
    write_goal_dumps=False,
    write_full_episode_dumps=False,
    render=False,
    write_video=False,
    dump_frequency=1,
    logdir='./',
    extra_players=None,
    number_of_left_players_agent_controls=1,
    number_of_right_players_agent_controls=1
)

In [24]:
def get_episodes(policy):
    
    episode_l = Episodes()
    episode_r = Episodes()

    obs_l, obs_r = env.reset()
    next_state_l = create_graph_from_observation(obs_l)
    next_state_r = create_graph_from_observation(obs_r)
    done = False
    
    while not done:
        state_l, state_r = next_state_l, next_state_r
        states = torch_geometric.data.Batch.from_data_list([
            state_l, state_r
        ])

        action_l, action_r = policy(states).sample().detach().numpy().tolist()
        #action_r = reverse_action(action_r_rev)

        (obs_l, obs_r), (reward_l, reward_r), done, info = env.step([action_l, action_r])

        next_state_l = create_graph_from_observation(obs_l)
        next_state_r = create_graph_from_observation(obs_r)

        episode_l.append(
            state=state_l,
            action=action_l,
            reward=reward_l,
            next_state=next_state_l,
            done=done,
        )
        episode_r.append(
            state=state_r,
            action=action_r,
            reward=reward_r,
            next_state=next_state_r,
            done=done,
        )
        
    return episode_l, episode_r

In [25]:
%%time
episodes = get_episodes(policy)

CPU times: user 1min, sys: 56.2 ms, total: 1min
Wall time: 1min


In [26]:
policy.eval()
episodes = Parallel(n_jobs=-1, backend='multiprocessing', verbose=100)([delayed(get_episodes)(policy) for _ in range(10)])

[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done   4 out of  10 | elapsed:  1.3min remaining:  1.9min
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:  2.4min remaining:  2.4min
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:  2.4min remaining:  1.6min
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:  2.4min remaining:  1.0min
[Parallel(n_jobs=-1)]: Done   8 out of  10 | elapsed:  2.4min remaining:   36.7s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  3.5min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  3.5min finished


In [ ]:
def agent(obs):
    global policy
    
    # Get observations for the first (and only one) player we control.
    #obs = Simple115StateWrapper.convert_observation(obs['players_raw'], True)
    graph = create_graph_from_observation(obs)
    actions = policy(graph)
    action = int(np.argmax(actions.detach().numpy()))
    return [action]

In [ ]:
def evaluate_agent(agent, env, n_times=50):
    scores = []
    for _ in tqdm(range(50)):
        obs = env.reset()
        rewards = []
        while True:
            action = agent([obs])
            obs, reward, done, info = env.step(action)
            rewards.append(reward)
            if done:
                break
        score = np.sum(rewards)
        print(score)
        scores.append(score)
    return np.mean(scores)

In [30]:
_episodes = episodes

In [34]:
episodes = []
for episode in _episodes:
    episodes.extend(episode)

In [36]:
concated_episode = concat_episodes(episodes)

In [39]:
sampled_episode = concated_episode.sample(10)

In [40]:
calculate_returns(sampled_episode, gamma=0.997,)

TypeError: calculate_returns() missing 2 required positional arguments: 'gamma' and 'value_net'

In [99]:
value = Value(num_features=9, num_relations=len(relations_dict))

In [100]:
batch = torch_geometric.data.Batch.from_data_list(sampled_episode['state'])

In [101]:
value(batch)

tensor([[0.0078],
        [0.0058],
        [0.0017],
        [0.0039],
        [0.0027],
        [0.0046],
        [0.0066],
        [0.0068],
        [0.0060],
        [0.0024]], grad_fn=<AddmmBackward>)

In [ ]:
calculate_returns()